# Encuesta Nacional de Nutrición y Salud (ENNyS2)

## Introducción

La Segunda Encuesta Nacional de Nutrición y Salud (ENNyS 2) fue realizada en Argentina entre octubre de 2018 y febrero de 2019 por el Ministerio de Salud.

Su objetivo es describir la situación alimentaria y de salud de la población, recolectando información sobre:

* Consumo de alimentos y bebidas (recordatorio de 24 horas, frecuencia y tipo de alimentos).

* Estado nutricional (mediciones antropométricas y composición corporal).

* Enfermedades crónicas y factores de riesgo (hipertensión, diabetes, colesterol alto, etc.).

* Condiciones sociodemográficas y del hogar (cobertura de salud, nivel educativo, ingresos, etc.).

* Hábitos de vida (actividad física, lactancia, consumo de tabaco y alcohol, etc.).

El diseño muestral es probabilístico, estratificado y multietápico, lo que asegura representatividad nacional y por regiones.

***

## Cuestionarios por grupo etario

* **C1:** Niños/as de 0 a 23 meses.

* **C2:** Niños/as de 2 a 12 años.

* **C3:** Adolescentes de 13 a 17 años.

* **C4:** Adultos de 18 años o más.

***

## Estructura de la base de la ENNyS2

La base principal tiene ~1830 columnas, organizadas en módulos y prefijos que identifican tipo de dato, población objetivo y, en algunos casos, la posición del miembro del hogar.

### Convenciones de nombres

* **C1\_ / C2\_ / C3\_ / C4\_**: variables de cuestionarios específicos según grupo etario.

* **T\_Cn\_**\*: totales o derivados de un módulo.

* **Mn\_**\*: información de miembros del hogar (hasta 15 posibles). Ejemplo: M01\_Edad, M02\_Sexo, M02\_Parentesco.

* **SD\_ / T\_SD\_**: sociodemográfico y derivados.

* **FCA\_**: frecuencia de consumo de alimentos.

* **HAC\_**: hábitos y conductas.

* **F\_**: factores de ponderación.

* **EE\_**: estados/estudios.

***

## Campos estructurales

Los campos clave para entender la jerarquía de la encuesta y vincular niveles (persona, hogar, vivienda, región) son:

* **id**\
  Identificador único de cada persona en la base (cada fila = una persona). Nunca se repite.

* **miembro\_id**\
  Identificador global de persona usado para vincular con otras bases (alimentos, nutrientes, antropometría).\
  Puede contener valores no numéricos en casos especiales (“No aplica”, “Si/No”), porque en la encuesta se usaron también como códigos de carga.

* **hogar\_id**\
  Agrupa a las personas que pertenecen al mismo hogar. Se repite en varias filas.\
  Ejemplo: hogar\_id = 38966 agrupa a 6 personas diferentes.

* **vivienda\_id**\
  Identifica la vivienda física. Puede contener uno o varios hogares.\
  Ejemplo: vivienda\_id = 820 incluye dos hogares distintos (hogar\_id 37529 y 94093).

* **region**\
  Área geográfica de la encuesta (Centro, NOA, Cuyo, Patagonia, etc.).\
  Algunos registros aparecen codificados con números en lugar de nombres.

* **ES\_Seleccionado**\
  Indica si la persona fue sorteada para responder el cuestionario individual completo (C1–C4).\
  Solo esos casos tienen datos de salud, antropometría y recordatorio de 24h.\
  Puede contener valores “Si”, “No”, “No aplica”, etc.

* **E\_CUEST**\
  Especifica cuál cuestionario se aplicó según edad (C1–C4).

***

## Columnas MXX\_

El bloque de columnas `M01_... M15_...` corresponde al **listado de convivientes (roster)** que el encuestador registró al inicio del cuestionario sociodemográfico. Allí se incluyen datos básicos (edad, sexo, parentesco, educación, etc.) de todos los miembros del hogar.

**Detalle importante:**

* En la base publicada, **ese bloque MX\_ está replicado en todas las filas del hogar**.

* Es decir: cada persona del hogar tiene su propio `id` (una fila) y además lleva consigo el mismo listado MX\_ completo.

* Por lo tanto, los MX\_ son útiles para reconstruir cómo fue la entrevista, pero redundan si ya trabajamos con las personas fila por fila.

***

## Cómo se realiza la encuesta

1. **Formulario sociodemográfico:**

   * Se registran todos los convivientes (edad, sexo, parentesco, educación, etc.) y condiciones de la vivienda.

   * Esta información queda en las columnas MX\_ y variables SD\_.

2. **Sorteo del individuo:**

   * Se elige aleatoriamente a un miembro del hogar.

   * Según la edad, responde el cuestionario C1–C4.

3. **Cuestionario individual (seleccionado):**

   * Solo el seleccionado tiene información completa: estado de salud, antecedentes, antropometría, recordatorio de 24h, etc.

4. **Resto de convivientes:**

   * También aparecen como filas con su `id`, pero solo con datos básicos sociodemográficos.

   * Sus módulos C1–C4 están vacíos.

***

## Procedimiento para el análisis

1. Separar el bloque MX\_ en una base aparte (opcional, porque ya está replicado en cada fila).

2. Filtrar a los seleccionados (`ES_Seleccionado = "Si"`) para análisis con cuestionarios completos.

3. Dividir en grupos etarios (C1–C4).

4. Analizar variables de interés (socioeconómicas, consumo, estado nutricional).

***

## 1. Creamos una base sin la columnas MX_ y una base con las columnas MX_

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Cargar base de la encuesta
DATA = Path("../data")  # tu carpeta de datos
FILE = DATA / "ENNyS2_encuesta.csv"

df = pd.read_csv(FILE, encoding="utf-8-sig", low_memory=False)
print("Base original:", df.shape)


# Identificadores que se deben conservar en ambas tablas
id_cols = ["id"]

# Selección exacta de columnas de miembros: M00_ .. M15_
pat = re.compile(r"^M(0[0-9]|1[0-5])_")
m_cols = [c for c in df.columns if pat.match(c)]

print("Columnas M00_..M15_ detectadas:", len(m_cols))

# Base sin columnas
df_sin_m = df.drop(columns=m_cols, errors="ignore").copy()

# Base solo con columnas Mxx + IDs 
df_solo_m = df[id_cols + m_cols].copy()

print("ENNyS2_sin_M:", df_sin_m.shape)
print("ENNyS2_solo_M:", df_solo_m.shape)

Base original: (21358, 1830)
Columnas M00_..M15_ detectadas: 510
ENNyS2_sin_M: (21358, 1320)
ENNyS2_solo_M: (21358, 511)


## Verificacion de la duplicacion de datos en los campos MX_

In [7]:
# --- Verificar duplicación de MX_ dentro de un mismo hogar/vivienda ---

# 1) elegí un hogar_id de ejemplo (de los que compartiste antes)
hogar_ej = 38966   # podés cambiar por 37529, 94093, etc.
vivienda_ej = 276  # o 820, etc.

# 2) subset de las filas de ese hogar/vivienda
subset = df[(df["hogar_id"] == hogar_ej) & (df["vivienda_id"] == vivienda_ej)]

# 3) seleccionar algunas columnas MX_ para chequear
mx_cols = [c for c in df.columns if c.startswith("M") and c[1:3].isdigit()]
sample_cols = mx_cols[:10]  # tomar solo las primeras 10 para mostrar

# 4) mostrar tabla reducida
print(subset[["id", "ES_Seleccionado"] + sample_cols].head())

# 5) verificar si todas las filas del hogar tienen exactamente el mismo bloque MX_
igual = subset[sample_cols].nunique().max() == 1
print(f"¿Todas las filas de hogar {hogar_ej} / vivienda {vivienda_ej} tienen los mismos MX_?: {igual}")



             id ES_Seleccionado M01_miembro_id M01_caso_id  \
522    81315828              Si         119128    81315828   
523    81315830              Si         119128    81315828   
11790  81315831              Si         119128    81315828   
11791  81315832  No aplica edad         119128    81315828   
11792  81315833  No aplica edad         119128    81315828   

                                M01_clave  M01_sd_caso_id   M01_SD_3  \
522    234/30/124/1/1/6/1248/13/47/111/11        81315828  Masculino   
523    234/30/124/1/1/6/1248/13/47/111/11        81315828  Masculino   
11790  234/30/124/1/1/6/1248/13/47/111/11        81315828  Masculino   
11791  234/30/124/1/1/6/1248/13/47/111/11        81315828  Masculino   
11792  234/30/124/1/1/6/1248/13/47/111/11        81315828  Masculino   

       M01_SD_4 M01_SD_5 M01_SD_6 M01_SD_10 M01_SD_11  
522          47            Jefe/a        No            
523          47            Jefe/a        No            
11790        47           


## 3. Separamos las 4 subbases (C1 a C4) en df_sin_m

In [3]:
df_sel = df_sin_m[df_sin_m["ES_Seleccionado"]=="Si"]

c1 = df_sel[df_sel["E_CUEST"]=="0 a 23 meses"]
c2 = df_sel[df_sel["E_CUEST"]=="2 a 12 años"]
c3 = df_sel[df_sel["E_CUEST"]=="13 a 17 años"]
c4 = df_sel[df_sel["E_CUEST"]=="18 o mas años"]

print("Seleccionados:", df_sel.shape[0])
print("0 a 23 meses:", c1.shape[0])
print("2 a 12 años:", c2.shape[0])
print("13 a 17 años:", c3.shape[0])
print("18 años y más:", c4.shape[0])

Seleccionados: 5506
0 a 23 meses: 2013
2 a 12 años: 216
13 a 17 años: 401
18 años y más: 2876


## 4. Guardamos los resultados

In [4]:
# --- Guardar ---
OUT = DATA / "processed"
OUT.mkdir(parents=True, exist_ok=True)
df_sin_m.to_csv(OUT / "ENNyS2_sin_M.csv", index=False)
df_solo_m.to_csv(OUT / "ENNyS2_solo_M.csv", index=False)
c1.to_csv(OUT / "ENNyS2_c1.csv", index=False)
c2.to_csv(OUT / "ENNyS2_c2.csv", index=False)
c3.to_csv(OUT / "ENNyS2_c3.csv", index=False)
c4.to_csv(OUT / "ENNyS2_c4.csv", index=False)


---

## 3. Cargamos base de alimentos

In [5]:
FILE = DATA / "Base_Alimentos_Bebidas_Suplementos.csv"

alimentos = pd.read_csv(FILE, encoding="utf-8-sig", low_memory=False) # Encoding para acentos

alimentos.shape

(26028, 1114)